In [61]:
import pandas as pd
import numpy as np
import requests
from datetime import datetime

api_url = "https://data.cityofnewyork.us/resource/fhrw-4uyv.json?"  #extracting the values for "complaint_types" column
query = (api_url +
        "&$select=complaint_type"
        "&$limit=500"
        "&$group=complaint_type"
        )

compl_type = requests.get(query)
# print compl_type.content
cols = 'agency,agency_name,borough,city,closed_date,complaint_type,created_date,descriptor,facility_type,incident_zip,latitude,longitude,park_borough,resolution_action_updated_date,resolution_description,status'
types = "'Fire Alarm - Addition','Fire Alarm - Modification','Fire Alarm - New System','Fire Alarm - Reinspection','Fire Alarm - Replacement'"

fire_query = "$select=" + cols + "%20&$where=complaint_type%20in(" + types.replace(' ','%20') + ")%20&$limit=500000%20"
fire_df = pd.read_json(api_url + fire_query)



In [63]:
working_df = pd.concat([fire_df.drop(['created_date','closed_date'], axis=1), fire_df.dropna()[['created_date', 'closed_date']].applymap(lambda x: datetime.strptime(x, '%Y-%m-%dT%H:%M:%S.000').date()) ], axis=1)

In [64]:
tmp1_df = working_df[['created_date', 'closed_date']].applymap(lambda x: pd.Timestamp(x).quarter)
tmp1_df.columns = ['quarter_created','quarter_closed']
tmp2_df = working_df[['created_date', 'closed_date']].applymap(lambda x: pd.Timestamp(x).year)
tmp2_df.convert_objects(convert_numeric=True)
tmp2_df.columns = ['year_created','year_closed']
working_df = pd.concat([working_df, tmp1_df, tmp2_df], axis=1, join='inner')
working_df['waiting_days']=(working_df['closed_date'] - working_df['created_date']).map(lambda x: x.astype('timedelta64[D]').astype(int))

In [65]:
grouped_df = pd.DataFrame({'count': working_df.dropna().groupby(['incident_zip','year_created','quarter_created','waiting_days']).waiting_days.count()}).reset_index()

In [66]:
from bokeh.charts import Bar, output_file, output_notebook, show
from bokeh.plotting import figure, output_file, show

In [73]:
p = Bar(grouped_df, label='year_created', values='waiting_days', agg='mean', stack='quarter_created',
        title="Number of 3-1-1 calls per year, stacked by quarter", legend='top_right') #agg='mean',

output_file("311fire_calls_bar.html")
#output_notebook()
show(p)

In [74]:
from bokeh.io import output_file, show, output_notebook
from bokeh.models import (
  GMapPlot, GMapOptions, ColumnDataSource, Circle, DataRange1d, PanTool, WheelZoomTool, BoxSelectTool
)

In [75]:
gsource = ColumnDataSource(data=dict(lat=fire_df['latitude'],lon=fire_df['longitude']))

In [76]:
#40°43′N 74°00′W rough placement of NYC

map_options = GMapOptions(lat=40.75, lng=-74.00, map_type="roadmap", zoom=11)

plot = GMapPlot(
    x_range=DataRange1d(), y_range=DataRange1d(), map_options=map_options, title="NY FIRE CALLS 311"
)

plot.plot_height = 1000
plot.plot_width = 1000

circle = Circle(x="lon", y="lat", size=5, fill_color="blue", fill_alpha=0.8, line_color=None)
plot.add_glyph(gsource, circle)

plot.add_tools(PanTool(), WheelZoomTool(), BoxSelectTool())

output_file("311fire_gmap_plot.html")
#output_notebook()
show(plot)